## Арсенин Никита, ЦМФ, 2021
#### Задание по предсказанию тренда цены биткоина по текстовым данным

In [5]:
import pandas as pd
import numpy as np
import math 

In [6]:
# Download data
data = pd.read_csv("cr.csv", error_bad_lines=False)
crypto = pd.read_csv('BTC.csv')

b'Skipping line 731: expected 8 fields, saw 13\nSkipping line 2836: expected 8 fields, saw 15\nSkipping line 3058: expected 8 fields, saw 12\nSkipping line 3113: expected 8 fields, saw 12\nSkipping line 3194: expected 8 fields, saw 17\nSkipping line 3205: expected 8 fields, saw 17\nSkipping line 3255: expected 8 fields, saw 17\nSkipping line 3520: expected 8 fields, saw 17\nSkipping line 4078: expected 8 fields, saw 17\nSkipping line 4087: expected 8 fields, saw 17\nSkipping line 4088: expected 8 fields, saw 17\nSkipping line 4499: expected 8 fields, saw 12\n'


In [7]:
# Clean up the data
data = data[["text", "timestamp"]]
data['timestamp'] = data['timestamp'].apply(lambda x: pd.Timestamp(x))
crypto['timestamp'] = crypto['Date'].apply(lambda x: pd.Timestamp(x))
crypto = crypto[['Close', 'timestamp']]

# We need only specific period of time
crypto = crypto.loc[10000:20000]

In [8]:
# Delete non-alphabletic characters from the string and split it into list of words
data['text'] = data['text'].apply(lambda s: ''.join(filter(lambda x: str.isalpha(x) or x==' ', s)))
data['list'] = data['text'].apply(lambda s: s.split(' '))
data = data[['timestamp', 'list']]

In [9]:
data

,timestamp,list
0,2018-07-18 21:33:26+00:00,"[VIDEO, I, was, in, my, office, I, was, mindin..."
1,2018-07-18 22:22:47+00:00,"[The, price, of, lumber, LBF, is, down, , sinc..."
2,2018-07-18 22:32:01+00:00,"[Who, says, the, American, Dream, is, dead, ht..."
3,2018-07-18 22:52:52+00:00,"[Barry, Silbert, is, extremely, optimistic, on..."
4,2018-07-18 23:00:01+00:00,"[How, satellites, avoid, attacks, and, space, ..."
...,...,...
28259,2018-07-18 23:46:13+00:00,"[FB, , acfdafcbaecf]"
28260,2018-07-18 23:46:19+00:00,"[仮想通貨ビットコインの価格上昇万円台回復約カ月半ぶりBTC, httpstcoTRIkYd..."
28261,2018-07-18 23:46:19+00:00,"[RT, investinhd, Nuff, said, , TEL, telcoin, T..."
28262,2018-07-18 23:46:20+00:00,"[仮想通貨ビットコインの価格上昇万円台回復約カ月半ぶりBTC, httpstcoOaMANO..."


In [10]:
crypto

,Close,timestamp
10000,3598.43,2019-02-17 05:00:00
10001,3595.80,2019-02-17 04:00:00
10002,3589.18,2019-02-17 03:00:00
10003,3586.31,2019-02-17 02:00:00
10004,3586.01,2019-02-17 01:00:00
...,...,...
19996,14889.34,2017-12-27 17:00:00
19997,14772.94,2017-12-27 16:00:00
19998,15295.16,2017-12-27 15:00:00
19999,15293.22,2017-12-27 14:00:00


In [11]:
# Compare two timestamps to be equal up to hour
def compare_timestamps(stamp1, stamp2):
    return ((stamp1.year == stamp2.year) and (stamp1.month == stamp2.month) and 
            (stamp1.day == stamp2.day) and (stamp1.hour == stamp2.hour))

In [12]:
# Calulate trend on given date
#    This function calculates the difference between current moment and future moment (over h hours)
#    then calculates standart devation of returns for previous n time values
#    returns both of these values
def calculate_trend(timestamp, crypto, n=5, h=6):
    main_stamp = crypto.loc[crypto['timestamp'].apply(lambda x: compare_timestamps(x, timestamp)) == True]
    main_index = main_stamp.index[0]
    var_stamps = crypto.loc[main_index:main_index+n]
    next_stamp = crypto.loc[main_index-h]
    var_diff = var_stamps['Close'].pct_change()
    var = var_diff.std()
    ret = (next_stamp['Close']-main_stamp['Close'][main_index])/main_stamp['Close'][main_index]
    return [ret, var]

In [54]:
# Calculate trend direction based on return/std ratio
def trend(x, cr1=3, cr2=-3):
    if x > cr1:
        return 1
    elif x < cr2:
        return -1
    return 0

In [14]:
# Calculates trends for each message in the dataset
# !!! Can take a long time !!!
dd = data.copy()
dd['temp'] = dd['timestamp']
dd['temp'] = dd['temp'].apply(lambda x: calculate_trend(x, crypto))

In [21]:
dd

,list,return,std,t,trend
0,"[VIDEO, I, was, in, my, office, I, was, mindin...",-0.002730,0.007792,-0.350418,0
1,"[The, price, of, lumber, LBF, is, down, , sinc...",-0.006009,0.005642,-1.065132,-1
2,"[Who, says, the, American, Dream, is, dead, ht...",-0.006009,0.005642,-1.065132,-1
3,"[Barry, Silbert, is, extremely, optimistic, on...",-0.006009,0.005642,-1.065132,-1
4,"[How, satellites, avoid, attacks, and, space, ...",-0.006770,0.004869,-1.390338,-1
...,...,...,...,...,...
28259,"[FB, , acfdafcbaecf]",-0.006770,0.004869,-1.390338,-1
28260,"[仮想通貨ビットコインの価格上昇万円台回復約カ月半ぶりBTC, httpstcoTRIkYd...",-0.006770,0.004869,-1.390338,-1
28261,"[RT, investinhd, Nuff, said, , TEL, telcoin, T...",-0.006770,0.004869,-1.390338,-1
28262,"[仮想通貨ビットコインの価格上昇万円台回復約カ月半ぶりBTC, httpstcoOaMANO...",-0.006770,0.004869,-1.390338,-1


In [16]:
# Calculates trends direction for each message in the dataset
dd['return'] = dd['temp'].apply(lambda x: x[0])
dd['std'] = dd['temp'].apply(lambda x: x[1])
dd = dd[['list', 'return', 'std']]
dd['t'] = dd['return']/dd['std']
dd['trend'] = dd['t'].apply(trend)

<ipython-input-16-539cd9ab9a07>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dd['t'] = dd['return']/dd['std']


In [56]:
# Select messages with positive and negative trend
positive = dd[dd['trend'] == 1].copy().reset_index()[['list', 't']]
negative = dd[dd['trend'] == -1].copy().reset_index()[['list', 't']]

In [57]:
positive

,list,t
0,"[RT, gassinCharts, TMT, , AEE, , POW, KRC, TNO...",3.328300
1,"[Q, , EPS, Estimates, for, Cimarex, Energy, Co...",3.328300
2,"[Pentair, Target, of, Unusually, Large, Option...",3.328300
3,"[Discovery, Inc, Series, C, Target, of, Unusua...",3.328300
4,"[Yum, Brands, Inc, YUM, Expected, to, Post, Qu...",3.328300
...,...,...
4081,"[Sign, up, and, get, , fee, discount, with, Bi...",3.106569
4082,"[RT, eWhispers, earnings, scheduled, before, t...",3.106569
4083,"[TRUMP, MAY, SUCCEED, SHOULD, YOU, BUY, OR, SE...",3.106569
4084,"[TRUMP, MAY, SUCCEED, SHOULD, YOU, BUY, OR, SE...",3.106569


In [58]:
uni_dict = {}
pos_dict = {}
neg_dict = {}

In [59]:
# Create dictionary for 
#    - words in all messages (uni_dict)
#    - words in positive messages (pos_dict)
#    - words in negaive messages (neg_dict)
for dicty in [(uni_dict, dd), (pos_dict, positive), (neg_dict, negative)]:
    for string in dicty[1]['list']:
        for w in string:
            word = w.lower()
            if word in dicty[0]:
                dicty[0][word] += 1
            else:
                dicty[0][word] = 1

In [60]:
# Calculate tf-idf for words in positive dataset (excluding short ones with length < 4) and sort 'em by 
#    - sqrt() was taken instead of log() cause it shows better perfomance
res_pos_dict = {}
for k, v in pos_dict.items():
    if len(k) < 4:
        continue
    res_pos_dict[k] = v/(1*math.sqrt(uni_dict[k]))

pos_sorted = {k: v for k, v in sorted(res_pos_dict.items(), key=lambda item: item[1], reverse=True)}
pos_sorted

{'earnings': 7.6659956764880945,
 'research': 7.028072659679503,
 'analysts': 6.9147905505668765,
 'stock': 6.77612631600587,
 'price': 6.734278701262227,
 'group': 6.663332499583072,
 'from': 6.017608255025234,
 'estimates': 5.918640302493727,
 'zacks': 5.833501953586106,
 'expected': 5.689138807086743,
 'rating': 5.646183764340528,
 'target': 5.604055438006111,
 'nflx': 5.578373255276352,
 'investment': 5.406667298997719,
 'previously': 5.3267004208738244,
 'short': 5.223329236878686,
 'high': 5.140135526227978,
 'financial': 5.076923076923077,
 'this': 4.8751780594262515,
 'head': 4.845551060443495,
 'hold': 4.841746452552157,
 'options': 4.83985287216211,
 'shares': 4.725949212549365,
 'class': 4.713772711872174,
 'week': 4.662019957299214,
 'energy': 4.656045901927967,
 'interest': 4.654919652649743,
 'million': 4.606269218712848,
 'share': 4.543627379421994,
 'stocks': 4.4578470987847565,
 'lowers': 4.3267007041221195,
 'international': 4.282930764670303,
 'sales': 4.277811855037

In [61]:
# Calculate tf-idf for words in negative dataset (excluding short ones with length < 4) and sort 'em by 
#    - sqrt() was taken instead of log() cause it shows better perfomance
res_neg_dict = {}
for k, v in neg_dict.items():
    if len(k) < 4:
        continue
    res_neg_dict[k] = v/(math.sqrt(uni_dict[k]))

neg_sorted = {k: v for k, v in sorted(res_neg_dict.items(), key=lambda item: item[1], reverse=True)}
neg_sorted

{'price': 7.186403419721528,
 'with': 6.61117417575996,
 'binance': 6.297250356557395,
 'today': 6.088590823425645,
 'this': 6.061032181989393,
 'from': 5.88207653756971,
 'stock': 5.674126534510106,
 'trading': 5.385718102125858,
 'chanos': 5.357061993998872,
 'earnings': 5.284547315448894,
 'stocks': 5.257185199256505,
 'eric': 5.159216173096193,
 'register': 5.128020396446639,
 'join': 5.008570288023919,
 'bonus': 5.002603488851246,
 'call': 4.930299750149778,
 'calls': 4.825693052074056,
 'market': 4.774934554525329,
 'invest': 4.732863826479693,
 'evhc': 4.638124095143555,
 'shares': 4.61934885437156,
 'siri': 4.562587456813871,
 'after': 4.488564149238307,
 'amat': 4.4692865530186845,
 'short': 4.468149106245623,
 'analysts': 4.446967774569511,
 'bullish': 4.443751166393376,
 'will': 4.363371598146444,
 'here': 4.257265482943858,
 'sbux': 4.233901974057256,
 'research': 4.224524549534127,
 'trade': 4.197098423810561,
 'target': 4.194446401452427,
 'closes': 4.184914994777494,
 'i